In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import accuracy_score,mean_squared_error,roc_curve,roc_auc_score,classification_report,r2_score,confusion_matrix

# Visualization Seaborn & Matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
# Plotly for interactive graphics 
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
#Disabling the warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/diabetes/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
cdf = df.copy()

In [ ]:
cdf.Pregnancies.value_counts()

In [ ]:
plt.figure(figsize = (12,6)) 
sns.heatmap(cdf.corr(),robust=True,fmt='.1g',linewidths=1.3,linecolor = 'gold', annot=True,);

In [ ]:
cdf.nunique()

In [ ]:
cdf.describe().T

In [ ]:
print("satir ve sutun=",cdf.shape)
print("boyut sayisi = ",cdf.ndim)
print("boyut sayisi = ",cdf.ndim)

In [ ]:
sns.countplot(cdf.Outcome);

In [ ]:
import missingno as msno
msno.matrix(cdf);

In [ ]:
x = cdf.drop(["Outcome"],axis = 1) #independent value

In [ ]:
y = cdf.Outcome

### Normalization

### PCA,Standart Scaler, Preprocessing etc.

In [ ]:
x = (x-np.min(x))/(np.max(x)-np.min(x)).values

In [ ]:
x.head()

## Train-test splitting

In [ ]:
from sklearn.model_selection import train_test_split,cross_val_score,ShuffleSplit,GridSearchCV
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2, random_state = 42)

### Model Selection

# 1) Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_reg = LogisticRegression().fit(x_train,y_train)
log_reg

In [ ]:
log_reg.intercept_
log_reg.coef_

In [ ]:
y_pred = log_reg.predict(x_test)
y_pred[0:10]

### Proba values - probability

In [ ]:
y_probs = log_reg.predict_proba(x_test)[:,1]

In [ ]:
y_pred = [1 if i >0.45 else 0 for i in y_probs]
y_pred[:10]

In [ ]:
log_score = accuracy_score(y_test,y_pred)
print ("log score=",log_score)

#### Accuracy Score

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
log_score = accuracy_score(y_test,y_pred)
print ("log score=",log_score)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
y_pred = [1 if i >0.52 else 0 for i in y_probs]
y_pred[:10]


## 2) Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
r_for = RandomForestClassifier().fit(x_train,y_train)
r_for

In [ ]:
y_pred = r_for.predict(x_test)

In [ ]:
rf_score = accuracy_score(y_test,y_pred)
rf_score

#### Importance

In [ ]:
Importance = pd.DataFrame({"Importance": r_for.feature_importances_*100},
                         index = x_train.columns)
Importance.sort_values(by = "Importance", 
                       axis = 0, 
                       ascending = True).plot(kind ="barh", color = "g")

plt.xlabel("Değişken Önem Düzeyleri")

## 3) KNN model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3).fit(x_train,y_train)
knn

In [ ]:
y_pred = knn.predict(x_test)

In [ ]:
knn_score = accuracy_score(y_test,y_pred)
knn_score

In [ ]:
confusion_matrix(y_test,y_pred)

### Model Tuning

In [ ]:
knn_params = {"n_neighbors":np.arange(1,50)}

In [ ]:
knn =KNeighborsClassifier()
knn_cv = GridSearchCV(knn,knn_params,cv = 10)
knn_cv = knn_cv.fit(x_train,y_train)

In [ ]:
print("Best Score:"+str(knn_cv.best_score_))
print("Best Parameters:"+str(knn_cv.best_params_))

In [ ]:
knn_final = KNeighborsClassifier(n_neighbors=1)
knn_final.fit(x_train,y_train)

In [ ]:
y_pred = knn_final.predict(x_test)
knn_fscore = accuracy_score(y_test,y_pred)
knn_fscore

In [ ]:
confusion_matrix(y_test,y_pred)

## 4) SVM - Support Vector Machine

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm_model = SVC(C=5,degree=9,kernel = 'poly').fit(x_train,y_train)
svm_model

In [ ]:
y_pred = svm_model.predict(x_test)
svm_score = accuracy_score(y_test,y_pred)
svm_score

In [ ]:
confusion_matrix(y_test,y_pred)

## 5) Gaussian Naive Bayes Model

In [ ]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb_model = nb.fit(x_train, y_train)
nb_model

In [ ]:
nb_model.predict(x_test)[:10]

In [ ]:
nb_model.predict_proba(x_test)[0:10]  


In [ ]:
y_pred = nb_model.predict(x_test)
nb_score = accuracy_score(y_test, y_pred)
print("NB_class_SCORE = ", nb_score)  

In [ ]:
cross_val_score(nb_model, x_test, y_test, cv = 20).mean()  

In [ ]:
indexx = ["Log","RF","KNN","SVM","NB"]
regressions = [log_score,rf_score,knn_fscore,svm_score,nb_score]

plt.figure(figsize=(8,6))
sns.barplot(x=indexx,y=regressions)
plt.xticks()
plt.title('Model Comparision',color = 'orange',fontsize=20);


In [ ]:
indexx = ["Log","RF","KNN","SVM","NB"]
regressions = [log_score,rf_score,knn_fscore,svm_score,nb_score]

import plotly.express as px

fig = px.pie(df, values=regressions, names=indexx, title='Regression Score Results')
fig.show()